# New metrics 

This notebook defines and implements the new metrics of my thesis for synthetic data assessment. 

## Packages and libraries mandatory

In [38]:
from shapely.geometry import Point, Polygon
import pandas as pd
import os 
import numpy as np
import os.path
from os import path as pa
import json
from tqdm.notebook import tqdm
import copy
from collections import Counter
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,FactorAnalysis
from scipy.stats import ks_2samp,chisquare
import torch
import sys 
sys.path.append('../torch-two-sample')
import json
from torch_two_sample import MMDStatistic, EnergyStatistic,FRStatistic,KNNStatistic
from scipy.spatial import Delaunay,ConvexHull
sys.path.append('../DataShapley')
from Shapley import ShapNN
from DShap import DShap
from shap_utils import *
from preferencefig import *
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial import ConvexHull


## M_FAMD

### Example with one dataset

#### Loading real and synthetic datasets

In [39]:
name_dataset = 'witmer_census_1980_1'   # TODO : put dataset name. Ensure the path to the real and synthetic datasets are correct
path_dataset = '../T_Data/'+name_dataset+'/Data/'+name_dataset
X_r = pd.read_csv(path_dataset+'.csv',index_col = [0])
X_s = pd.read_csv(path_dataset+'_GC.csv',index_col = [0])

#### Perform PCA FAMD
r_points and s_points are 2D array representing the projected real and synthetic samples

In [40]:
fa_r = FactorAnalysis(n_components=2)
if len(X_r.columns)>2 :
    r_points = fa_r.fit_transform(X_r)
    s_points = fa_r.transform(X_s)

#### Compute Intersection Area

In [41]:
k_r = ConvexHull(r_points)
k_s = ConvexHull(s_points)

x_r_1 = r_points[k_r.vertices,0]
y_r_1 = r_points[k_r.vertices,1]
x_r_1 = np.append(x_r_1,x_r_1[0])
y_r_1 = np.append(y_r_1,y_r_1[0])
area_r = Polygon(r_points[k_r.vertices]).area
area_s = Polygon(s_points[k_s.vertices]).area

eq_r = k_r.equations

x_s_1 = s_points[k_s.vertices,0]
y_s_1 = s_points[k_s.vertices,1]
x_s_1 = np.append(x_s_1,x_s_1[0])
y_s_1 = np.append(y_s_1,y_s_1[0])

eq_s = k_s.equations
eq_r_2 = np.stack((-eq_r[:,0]/eq_r[:,1],-eq_r[:,2]/eq_r[:,1]),axis=1)
eq_s_2 = np.stack((-eq_s[:,0]/eq_s[:,1],-eq_s[:,2]/eq_s[:,1]),axis=1)

x_intersection = np.zeros((len(eq_r_2)*len(eq_s_2))) 
y_intersection = np.zeros((len(eq_r_2)*len(eq_s_2))) 

for i in range(len(eq_r_2)) :
    for j in range(len(eq_s_2)) : 
        x_intersection[i*len(eq_s_2)+j] = (eq_r_2[i,1]-eq_s_2[j,1])/(eq_s_2[j,0]-eq_r_2[i,0])
        y_intersection[i*len(eq_s_2)+j] = eq_r_2[i,0]*x_intersection[i*len(eq_s_2)+j]+eq_r_2[i,1]
import matplotlib.path as mplPath
p_r = mplPath.Path(r_points[k_r.vertices])
p_s = mplPath.Path(s_points[k_s.vertices])
points = np.stack((x_intersection,y_intersection),axis=1)
m1 = p_r.contains_points(points,radius = 0.00001)
m2 = p_s.contains_points(points,radius = 0.00001)
m3 = 1*m1+1*m2
ind_int = np.where(m3==2)[0]
x_int = points[ind_int,0]
y_int = points[ind_int,1]
mask_4 = np.zeros(len(x_r_1))
mask_5 = np.zeros(len(x_s_1))

point_r = np.stack((x_r_1,y_r_1),axis=1)
point_s = np.stack((x_s_1,y_s_1),axis=1)
m4 = p_r.contains_points(point_s,radius = 0.00001)
m5 = p_s.contains_points(point_r,radius = 0.00001)
point_r_int = point_s[m4]
point_s_int = point_r[m5]

x_f = np.concatenate((x_int,point_s_int[:,0],point_r_int[:,0]))
y_f = np.concatenate((y_int,point_s_int[:,1],point_r_int[:,1]))
p_int = np.stack((x_f,y_f),axis=1)
k_int = ConvexHull(p_int)
int_area = Polygon(p_int[k_int.vertices,:]).area
x_f2 = x_f[k_int.vertices]
y_f2 = y_f[k_int.vertices]
x_f2 = np.append(x_f2,x_f2[0])
y_f2 = np.append(y_f2,y_f2[0])

union_point = np.concatenate((r_points,s_points),axis=0)
k_union = ConvexHull(union_point)
union_area = Polygon(union_point[k_union.vertices]).area
m_famd = int_area/union_area



#### Compute Union area
m_famd is the metric score.

In [42]:
union_point = np.concatenate((r_points,s_points),axis=0)
k_union = ConvexHull(union_point)
union_area = Polygon(union_point[k_union.vertices]).area
m_famd = int_area/union_area

#### Get graph

In [43]:
fig,ax = plt.subplots()
ax.plot(r_points[:,0],r_points[:,1],'r.')
ax.plot(s_points[:,0],s_points[:,1],'b.')
ax.plot(x_s_1,y_s_1,'b-')
ax.plot(x_r_1,y_r_1,'r-')
ax.plot(x_f2,y_f2,'g-')
ax.plot(x_int,y_int,'+')
ax.grid(True)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title('$M_{FAMD} $ Score : '+ str(np.round(m_famd,2)))
fig.savefig('../Results/Result_11/Figures/PCA_1.jpeg',dpi=250)

### Method to compute the M_FAMD

In [2]:
def compute_pca_met(X_r,X_s):
    '''
    Method to compute M_FMAD metric.
    First, the 2 prinicpal components are fitted.
    Then, real and synthetic are projected according the two principal components. This gives real and synthetic data 2D scatter plots.
    From the scatter plots, intersection and union area are computed. 
    Finally, the metric is the ratio between the intersection and the union.  

    Input :

    X_r : pandas dataframe, real dataset.
    X_s : pandas dataframe, synthetic dataset.

    Output : 

    m_famd : float, metric score

    '''

    fa_r = FactorAnalysis(n_components=2)
    if len(X_r.columns)>2 :
        r_points = fa_r.fit_transform(X_r)
        s_points = fa_r.transform(X_s)
    from shapely.geometry import Point, Polygon
    k_r = ConvexHull(r_points)
    k_s = ConvexHull(s_points)
    
    x_r_1 = r_points[k_r.vertices,0]
    y_r_1 = r_points[k_r.vertices,1]
    x_r_1 = np.append(x_r_1,x_r_1[0])
    y_r_1 = np.append(y_r_1,y_r_1[0])
    area_r = Polygon(r_points[k_r.vertices]).area
    area_s = Polygon(s_points[k_s.vertices]).area

    eq_r = k_r.equations

    x_s_1 = s_points[k_s.vertices,0]
    y_s_1 = s_points[k_s.vertices,1]
    x_s_1 = np.append(x_s_1,x_s_1[0])
    y_s_1 = np.append(y_s_1,y_s_1[0])

    eq_s = k_s.equations
    eq_r_2 = np.stack((-eq_r[:,0]/eq_r[:,1],-eq_r[:,2]/eq_r[:,1]),axis=1)
    eq_s_2 = np.stack((-eq_s[:,0]/eq_s[:,1],-eq_s[:,2]/eq_s[:,1]),axis=1)

    x_intersection = np.zeros((len(eq_r_2)*len(eq_s_2))) 
    y_intersection = np.zeros((len(eq_r_2)*len(eq_s_2))) 

    for i in range(len(eq_r_2)) :
        for j in range(len(eq_s_2)) : 
            x_intersection[i*len(eq_s_2)+j] = (eq_r_2[i,1]-eq_s_2[j,1])/(eq_s_2[j,0]-eq_r_2[i,0])
            y_intersection[i*len(eq_s_2)+j] = eq_r_2[i,0]*x_intersection[i*len(eq_s_2)+j]+eq_r_2[i,1]
    import matplotlib.path as mplPath
    p_r = mplPath.Path(r_points[k_r.vertices])
    p_s = mplPath.Path(s_points[k_s.vertices])
    points = np.stack((x_intersection,y_intersection),axis=1)
    m1 = p_r.contains_points(points,radius = 0.00001)
    m2 = p_s.contains_points(points,radius = 0.00001)
    m3 = 1*m1+1*m2
    ind_int = np.where(m3==2)[0]
    x_int = points[ind_int,0]
    y_int = points[ind_int,1]
    mask_4 = np.zeros(len(x_r_1))
    mask_5 = np.zeros(len(x_s_1))

    point_r = np.stack((x_r_1,y_r_1),axis=1)
    point_s = np.stack((x_s_1,y_s_1),axis=1)
    m4 = p_r.contains_points(point_s,radius = 0.00001)
    m5 = p_s.contains_points(point_r,radius = 0.00001)
    point_r_int = point_s[m4]
    point_s_int = point_r[m5]

    x_f = np.concatenate((x_int,point_s_int[:,0],point_r_int[:,0]))
    y_f = np.concatenate((y_int,point_s_int[:,1],point_r_int[:,1]))
    p_int = np.stack((x_f,y_f),axis=1)
    k_int = ConvexHull(p_int)
    int_area = Polygon(p_int[k_int.vertices,:]).area
    x_f2 = x_f[k_int.vertices]
    y_f2 = y_f[k_int.vertices]
    x_f2 = np.append(x_f2,x_f2[0])
    y_f2 = np.append(y_f2,y_f2[0])
    
    area_int = k_int.area

    union_point = np.concatenate((r_points,s_points),axis=0)
    k_union = ConvexHull(union_point)
    union_area = Polygon(union_point[k_union.vertices]).area
    m_famd = int_area/union_area
    return m_famd

## Statistical metrics

### Check number of permutation for permutation method

In [ ]:
def compute_stat_test(X_r,X_s,names,num_perm=400):

    '''
    Method to compute new multivariate two samples statistical tests.

    Inputs : 

    X_r = panda dataframe, real dataset.
    X_s = panda dataframe, synthetic dataset. 

    names : list of string, list of the statistical test perform. 
            The different tests are 'ENE', 'KNN', 'MMD', 'FR'
    
    num_perm : integer, number of permutation for the permutation method to compute the p-value of the tests.

    Output : 

    score : dictionary, the key is the name of the test and the value its score.

    '''


    X_r = torch.tensor(X_r.values)
    X_s = torch.tensor(X_s.values)
    result = np.zeros(num_perm)
    result_2 = np.zeros(num_perm)
    size = int(0.5*len(X_r))
    if len(X_r)%2==0 :
        imp = False 
        n_1 = size
        n_2 = size
    else :
        imp = True
        n_1 = size
        n_2 = size+1
    dict_test = {'FR':FRStatistic(n_1,n_2),'KNN':KNNStatistic(n_1,n_2,5),'MMD':MMDStatistic(n_1,n_2),'ENE':EnergyStatistic(n_1,n_2)}
    score = {}
    
    for n in names : 
        FRR = dict_test[n]
        for i in range(num_perm): 
            mask = torch.randperm(len(X_r))
            mask2 = torch.randperm(len(X_s))
            
            X_r1 = X_r[mask[:size],:]
            X_r2 = X_r[mask[size:],:]
            if not imp : 
                X_s2 = X_s[mask2[:size],:]
            else :
                X_s2 = X_s[mask2[:size+1],:]
            if n == 'MMD' : 
                result[i] = FRR(X_r1,X_r2,alphas=[1,1],ret_matrix=False)
                result_2[i] = FRR(X_r1,X_s2,alphas=[1,1],ret_matrix=False)
            else :
                result[i] = FRR(X_r1,X_r2,ret_matrix=False)
                result_2[i] = FRR(X_r1,X_s2,ret_matrix=False)
        b_s = result_2.mean()
        pvalue = np.sum(b_s<result)/num_perm
        m = (1-2*0.1)/(0.1**2)
        score[n] = np.log(m*pvalue+1)/np.log(m+1)
    return score

## M_SHAP metric

In [ ]:
def compute_M_shap(X_r,X_test,X_s) : 
    ''' 
    Method to compute M_SHAP metric.


    Inputs : 
    
    X_r  : panda dataframe, real dataset use in the training.
    X_test : panda dataframe, real dataset used for testing.
    X_s : panda dataframe, synthetic dataset. 

    Output : 

    score : float, M_SHAP.
    
    '''
    problem, model = 'classification', 'logistic'
    X = pd.concat([X_r,X_s],axis=0,ignore_index=True)
    y_test = X_test['class'].values.astype('int')
    X_test = X_test.drop('class',axis=1)
    X_test = X_test.values
    y = X['class'].values.astype('int')
    X = X.drop('class',axis=1)
    X = X.values
    num_test = 1000
    directory2 = 'Shapley/Exp_1/' #'../Result_03/Data/Shapley_3/'
    dshap = DShap(X, y, X_test, y_test, num_test, 
                sources=None, 
                sample_weight=None,
                model_family=model, 
                metric='accuracy',
                overwrite=True,
                directory=directory2, seed=0)
    dshap.run(100, 0.1, g_run=False)
    vals_tmc_good = pd.Series(dshap.vals_tmc)
    v1 = vals_tmc_good
    m_1 = v1[v1.index<int(0.5*len(X))]
    m_2 = v1[v1.index>int(0.5*len(X))]
    min_t = np.minimum(np.min(m_1),np.min(m_2))
    max_t = np.maximum(np.max(m_1),np.max(m_2))
    m_11s = (m_1-min_t)/(max_t-min_t)
    m_22s = (m_2-min_t)/(max_t-min_t)
    m2_m = m_22s.mean()
    m1_m = m_11s.mean()
    mm = (1-2*m1_m)/(m1_m**2)
    score = np.log(mm*m2_m+1)/np.log(mm+1)
    return score